In [43]:
# Import and init h2o lib. Running on java
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,17 hours 42 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_paaske_rcki3a
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.881 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [45]:
# Reading csv to h2o dataframe 
col_types_train = {'store_id': 'enum',
 'chain_name': 'enum',
 'mall_name': 'enum',
 'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'revenue': 'int',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'int',
 'counts_gr_lv2': 'int',
 'counts_municipality_lv2': 'int',
 'busstops_within_50m': 'int',
 'busstops_within_100m': 'int',
 'busstops_within_400m': 'int',
 'busstops_within_800m': 'int',
 'busstops_within_1500m': 'int',
 'num_stores_within_100m_and_same_lvl2': 'int',
 'num_stores_within_500m_and_same_lvl2': 'int',
 'num_stores_within_1km_and_same_lvl2': 'int',
 'num_stores_within_5km_and_same_lvl2': 'int',
 'num_stores_within_10km_and_same_lvl2': 'int',
 'num_stores_within_20km_and_same_lvl2': 'int'}

col_types_test = {'store_id': 'enum',
 'chain_name': 'enum',
 'mall_name': 'enum',
 'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'int',
 'counts_gr_lv2': 'int',
 'counts_municipality_lv2': 'int',
 'busstops_within_50m': 'int',
 'busstops_within_100m': 'int',
 'busstops_within_400m': 'int',
 'busstops_within_800m': 'int',
 'busstops_within_1500m': 'int',
 'num_stores_within_100m_and_same_lvl2': 'int',
 'num_stores_within_500m_and_same_lvl2': 'int',
 'num_stores_within_1km_and_same_lvl2': 'int',
 'num_stores_within_5km_and_same_lvl2': 'int',
 'num_stores_within_10km_and_same_lvl2': 'int',
 'num_stores_within_20km_and_same_lvl2': 'int'}

 
train_data = h2o.import_file('feature_data/training_set_dropped.csv', col_types = col_types_train)
test_data = h2o.import_file('feature_data/testing_set_dropped.csv', col_types = col_types_test)

train_data.types
test_data.types

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


{'store_id': 'string',
 'chain_name': 'string',
 'mall_name': 'string',
 'lv1': 'string',
 'lv2': 'string',
 'lv3': 'string',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'real',
 'counts_gr_lv2': 'int',
 'counts_municipality_lv2': 'int',
 'busstops_within_50m': 'int',
 'busstops_within_100m': 'int',
 'busstops_within_400m': 'int',
 'busstops_within_800m': 'int',
 'busstops_within_1500m': 'int',
 'num_stores_within_100m_and_same_lvl2': 'int',
 'num_stores_within_500m_and_same_lvl2': 'int',
 'num_stores_within_1km_and_same_lvl2': 'int',
 'num_stores_within_5km_and_same_lvl2': 'int',
 'num_stores_within_10km_and_same_lvl2': 'int',
 'num_stores_within_20km_and_same_lvl2': 'int'}

In [46]:
# Setting features as factor
""" train_data['store_id'] = train_data['store_id'].asfactor()
train_data['chain_name'] = train_data['chain_name'].asfactor()
train_data['mall_name'] = train_data['mall_name'].asfactor()
train_data['revenue'] = train_data['revenue'].asfactor()
train_data['lv1'] = train_data['lv1'].asfactor()
train_data['lv2'] = train_data['lv2'].asfactor()
train_data['lv3'] = train_data['lv3'].asfactor()
train_data['total_nbr_people'] = train_data['total_nbr_people'].asfactor()
train_data['nbr_people_per_store_in_grunnkrets'] = train_data['nbr_people_per_store_in_grunnkrets'].asfactor()
train_data['counts_gr_lv2'] = train_data['counts_gr_lv2'].asfactor()
train_data['counts_municipality_lv2'] = train_data['counts_municipality_lv2'].asfactor()
train_data['busstops_within_50m'] = train_data['busstops_within_50m'].asfactor()
train_data['busstops_within_100m'] = train_data['busstops_within_100m'].asfactor()
train_data['busstops_within_400m'] = train_data['busstops_within_400m'].asfactor()
train_data['busstops_within_800m'] = train_data['busstops_within_800m'].asfactor()
train_data['busstops_within_1500m'] = train_data['busstops_within_1500m'].asfactor()
train_data['num_stores_within_100m_and_same_lvl2'] = train_data['num_stores_within_100m_and_same_lvl2'].asfactor()
train_data['num_stores_within_500m_and_same_lvl2'] = train_data['num_stores_within_500m_and_same_lvl2'].asfactor()
train_data['num_stores_within_1km_and_same_lvl2'] = train_data['num_stores_within_1km_and_same_lvl2'].asfactor()
train_data['num_stores_within_5km_and_same_lvl2'] = train_data['num_stores_within_5km_and_same_lvl2'].asfactor()
train_data['num_stores_within_10km_and_same_lvl2'] = train_data['num_stores_within_10km_and_same_lvl2'].asfactor()
train_data['num_stores_within_20km_and_same_lvl2'] = train_data['num_stores_within_20km_and_same_lvl2'].asfactor() """

H2OValueError: Only 'int' or 'string' are allowed for asfactor(), got revenue:real 

In [ ]:
#test_data['revenue'] = test_data['revenue'].asfactor()
""" test_data['store_id'] = test_data['store_id'].asfactor()
test_data['chain_name'] = test_data['chain_name'].asfactor()
test_data['mall_name'] = test_data['mall_name'].asfactor()
test_data['lv1'] = test_data['lv1'].asfactor()
test_data['lv2'] = test_data['lv2'].asfactor()
test_data['lv3'] = test_data['lv3'].asfactor()
test_data['total_nbr_people'] = test_data['total_nbr_people'].asfactor()
test_data['nbr_people_per_store_in_grunnkrets'] = test_data['nbr_people_per_store_in_grunnkrets'].asfactor()
test_data['counts_gr_lv2'] = test_data['counts_gr_lv2'].asfactor()
test_data['counts_municipality_lv2'] = test_data['counts_municipality_lv2'].asfactor()
test_data['busstops_within_50m'] = test_data['busstops_within_50m'].asfactor()
test_data['busstops_within_100m'] = test_data['busstops_within_100m'].asfactor()
test_data['busstops_within_400m'] = test_data['busstops_within_400m'].asfactor()
test_data['busstops_within_800m'] = test_data['busstops_within_800m'].asfactor()
test_data['busstops_within_1500m'] = test_data['busstops_within_1500m'].asfactor()
test_data['num_stores_within_100m_and_same_lvl2'] = test_data['num_stores_within_100m_and_same_lvl2'].asfactor()
test_data['num_stores_within_500m_and_same_lvl2'] = test_data['num_stores_within_500m_and_same_lvl2'].asfactor()
test_data['num_stores_within_1km_and_same_lvl2'] = test_data['num_stores_within_1km_and_same_lvl2'].asfactor()
test_data['num_stores_within_5km_and_same_lvl2'] = test_data['num_stores_within_5km_and_same_lvl2'].asfactor()
test_data['num_stores_within_10km_and_same_lvl2'] = test_data['num_stores_within_10km_and_same_lvl2'].asfactor()
test_data['num_stores_within_20km_and_same_lvl2'] = test_data['num_stores_within_20km_and_same_lvl2'].asfactor() """

In [ ]:
# Removing labels from train set
y = "revenue"
x = train_data.columns
x.remove(y)

In [ ]:
# Running fit/training on train set with cross validation n=5
aml = H2OAutoML(max_models = 10, seed = 1)
aml.train(x = x, y = y, training_frame = train_data)

In [ ]:
lb = aml.leaderboard

In [ ]:
# Showing best perforimg models
lb.head()

In [ ]:
preds = aml.predict(test_data)

In [ ]:
preds = h2o.as_list(preds)

In [ ]:
# Convert result to desired format

import pandas as pd
predictions = preds

predictions["sample_id"] = predictions.index + 50000
predictions["prediction_in_probability"] = predictions["p1"]
predictions[["sample_id", "prediction_in_probability"]].to_csv("predictions.txt", index=False)